# Dissolution Stability Machine Learning Project Using a Gaussian Mixture Model

## Project Overview – Analyzing Dissolution Test Data with Gaussian Mixture Models

**Goal:**  
Pharmaceutical tablets often dissolve at different rates depending on how they were manufactured (e.g., compression force, particle size). Sometimes, two batches made the same way can behave differently — suggesting hidden sub-populations. FDA Dissolution Methods metadata + synthetic dissolution profiles may be used to identify clusters of dissolution behaviors ("fast," "medium," "slow"). This project aims to use a Gaussian Mixture Model (GMM) to check if the data might come from multiple distributions (e.g., “fast” and “slow” dissolving tablets).

**Steps:**  
1. Feature Engineering from FDA database  
2. Synthetic Profile Generation using kinetic models  
3. GMM clustering and visualization  
4. Interpretation & discussion of pharma relevance

### 1. Notebook Setup

Getting all the tools ready in the project's virtual environment.

In [1]:
# # OPTIONAL: Check to review directory of functions available in preprocessing.py
# import preprocessing
# print(dir(preprocessing))

In [2]:
# Setup matplotlib to plot inline (within the notebook)
%matplotlib inline

# Core libraries
import pandas as pd
import numpy as np
import re

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# # Modeling
# from sklearn.preprocessing import StandardScaler
# from sklearn.mixture import GaussianMixture
# from sklearn.metrics import silhouette_score

# Utility
from tqdm import tqdm

# Functions from preprocessing.py
from preprocessing import (
    load_fda_data,
    clean_dosage_form_data,
    clean_apparatus_column,
    # extract_rpm_speeds,
    extract_medium_type,
    extract_medium_volumes,
    extract_sampling_times,
    simulate_dissolution_curve
)

# Ensure pandas prints enough rows and columns without truncation
pd.set_option("display.max_rows", 200)        # show up to 100 rows
pd.set_option("display.max_columns", None)    # show all columns
pd.set_option("display.width", None)          # don't wrap to fit terminal width
pd.set_option("display.max_colwidth", None)   # show full cell content

### 2. Data Acquisition

The FDA Dissolution Methods Database () contains:
- Dosage form
- Apparatus type
- Agitation speed (RPM)
- Medium type & volume
- Sampling times

These features can be parsed into a structured table for feature engineering.

Loading the dataset (if dataset is real and available).

In [3]:
# Import dataset from CSV file or URL
df = pd.read_csv("Dissolution Methods.csv")

# Quick check to view raw dataset size
print(len(df))

1568


In [4]:
# Quick check to view the dataset info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1568 entries, 0 to 1567
Data columns (total 8 columns):
 #   Column                                Non-Null Count  Dtype 
---  ------                                --------------  ----- 
 0   Drug Name                             1568 non-null   object
 1   Dosage Form                           1568 non-null   object
 2   USP Apparatus                         805 non-null    object
 3   Speed (RPMs)                          800 non-null    object
 4   Medium                                1568 non-null   object
 5   Volume (mL)                           792 non-null    object
 6   Recommended Sampling Times (minutes)  803 non-null    object
 7   Date Updated                          1568 non-null   object
dtypes: object(8)
memory usage: 98.1+ KB


In [5]:
# Quick check to view the dataset presentation
df.head()

,Drug Name,Dosage Form,USP Apparatus,Speed (RPMs),Medium,Volume (mL),Recommended Sampling Times (minutes),Date Updated
0,Abacavir Sulfate,Tablet,NaN,NaN,"Refer to FDA's Dissolution Guidance, 2018",NaN,NaN,07/02/2020
1,Abacavir Sulfate/Dolutegravir Sodium/Lamivudine,Tablet,II (Paddle),85,"0.01 M Phosphate Buffer with 0.5% sodium dodecyl sulfate (SDS), pH 6.8",900,"Abacavir and lamivudine: 10, 15, 20, 30 and 45; Dolutegravir: 5,15, 25, 35 and 45.",05/28/2015
2,Abacavir Sulfate/Dolutegravir Sodium/Lamivudine,Tablet (For Suspension),II (Paddle),50,"0.01 M Phosphate Buffer with 0.5 mM EDTA, pH 6.8",500,"5, 10, 15, 30, 45 and 60",10/06/2023
3,Abacavir Sulfate/Lamivudine,Tablet,II (Paddle),75,0.1 N HCl,900,"10, 20, 30, and 45",01/03/2007
4,Abacavir Sulfate/Lamivudine/Zidovudine,Tablet,II (Paddle),75,0.1 N HCl,Acid Stage: 900 mL; Buffer Stage: 1000 mL,"5, 10, 15, 30 and 45",01/03/2007


Simulating the dataset (if dataset if not available). (OPTIONAL)

In [6]:
# # Simulating dissolution test metrics for 200 batches
# np.random.seed(42)
# fast_group = np.random.normal(
#     loc=90, scale=5, size=(100, 3))  # high % dissolved
# slow_group = np.random.normal(
#     loc=60, scale=5, size=(100, 3))  # lower % dissolved
# synthetic_data = np.vstack([fast_group, slow_group])

# df = pd.DataFrame(synthetic_data, columns=["% Dissolved @ 5min", "% Dissolved @ 10min", "% Dissolved @ 15min"])

### 3. Data Cleaning, Preprocessing, and Feature Engineering

Prepare the dataset for use within the model. Convert the FDA method metadata into ML-ready features:

- Apparatus → categorical → one-hot encode  
- RPM → numerical
- Medium type → categorical → one-hot encode  
- Medium volume → numerical  
- Sampling times → numerical summary features (e.g., # of samples, max time)  

##### 3.1. Dosage Forms

The raw "Dosage Form" data entries can be separated into "Dosage Form" and "Release Type" columns. This may help the Gaussian Mixture Model (GMM) and the synthetic dissolution simulation to model broad dissolution behavior patterns as opposed to ultra-specific lab setups.

In [7]:
# Apply and expand "Dosage Form" data into two new columns
df[["DosageForm_Cleaned", "ReleaseType_Cleaned"]] = df["Dosage Form"].apply(lambda x: pd.Series(clean_dosage_form_data(x)))

# # Check results
print(df['DosageForm_Cleaned'].value_counts())
print(df['ReleaseType_Cleaned'].value_counts())

DosageForm_Cleaned
Tablet                                                971
Capsule                                               335
Suspension                                             67
Oral Suspension                                        17
Film, Transdermal                                      14
For Suspension                                         11
Ophthalmic Suspension                                   9
Film                                                    8
Transdermal                                             8
Injectable Suspension                                   8
Injectable                                              7
For Oral Suspension                                     6
Granule                                                 6
Implant                                                 6
Suppository                                             5
Gel                                                     4
Vaginal Insert                                       

##### 3.2. Apparatus Types

Only the main apparatus types should be kept. This is because, for the Gaussian Mixture Model (GMM) and synthetic dissolution simulation, the aim is to try to model broad dissolution behavior patterns instead of ultra-specific lab setups. Hence, main apparatus categories provide sufficient information to meaningfully model different dissolution behaviors.

Paddle and Basket and Paddle systems (I and II, respectively) represent fast and slow dissolution dynamics, while the others (III–VII) could be kept but are too rare to affect clustering.

**Note:**
- Apparatus I: "Basket"
- Apparatus II: "Paddle"
- Apparatus III: "Reciprocating Cylinder"
- Apparatus IV: "Flow-through Cell"
- Apparatus V: "Paddle over Disk"
- Apparatus VI: "Cylinder"
- Apparatus VII: "Reciprocating Holder"

In [8]:
df = clean_apparatus_column(df)
df['Apparatus_Cleaned'].value_counts(dropna=False)

Apparatus_Cleaned
NaN                       764
Paddle                    612
Basket                    143
Paddle over Disk           16
Cylinder                   10
Reciprocating Cylinder     10
Flow-through Cell           6
None                        4
Reciprocating Holder        3
Name: count, dtype: int64

In [9]:
616+ 13+ 11+ 9+2

651

In [10]:
# Basket
109+ 10 +5+ 2+ 2+ 1+ 1+ 1+ 1+ 1+ 1+ 1+ 1+ 1+ 1+ 1+ 1+ 1+ 1+ 1+ 1

144

In [11]:
# Paddles
474 +42+ 8+ 42+ 6+ 5+ 5+ 4+ 3+ 2+ 2+ 2+ 1+ 1+ 1+ 1+ 1+ 1+ 1+ 1+ 1+ 1+ 1+ 1+ 1+ 1+ 1

610

In [12]:
# Reciprocating Cylinder
6 +1+ 1+ 1+ 1

10

In [13]:
# Flow-through Cell
1 +1+ 1+ 1+ 1+ 1

6

In [14]:
# Paddle over Disk
2+ 2+ 2+ 2+ 1+ 1+ 1+ 1+ 1+ 1+ 1+ 1+ 1+ 1

18

In [15]:
# Cylinder
4 +1+ 1+ 1+ 1+ 1+ 1

10

In [16]:
# Reciprocating Holder
1 +1 +1

3

In [17]:
610+10+18+10+3

651

##### 3.3. Agitation Speeds

Only a single numeric RPM (or equivalent agitation speed) is needed per row. This value will be used later as a model input for simulating synthetic profile generation, so consistency and numeric type matter far more than preserving any additional textual detail.

In [18]:
df[df.columns[3]].value_counts()

Speed (RPMs)
50                                                                327
75                                                                209
100                                                               158
60                                                                 16
25                                                                 15
150                                                                 7
70                                                                  4
65                                                                  3
40                                                                  3
200                                                                 2
35                                                                  2
20 dpm                                                              2
55                                                                  2
175                                                                 1
RIB: 50

##### 3.4. Medium Types

Only the main medium types should be kept since the aim is still to try to model broad dissolution behavior patterns instead of ultra-specific lab setups. Hence, main medium categories provide enough information to model different dissolution behaviors.

In [19]:
pd.set_option("display.max_rows", None)  # show all rows
# df[df.columns[4]].value_counts()

In [20]:
# Apply the function to your DataFrame
df["MediumType_Cleaned"] = df["Medium"].apply(extract_medium_type)

# Explode the list into separate rows
df = df.explode("MediumType_Cleaned", ignore_index=True)

# # Now, you can use value_counts() to see the distribution
# print(df["MediumType_Cleaned"].value_counts())

##### 3.5. Medium Volumes

After the main medium types have been determined, the same can be done to determine the main medium volumes.

In [21]:
# df[df.columns[5]].value_counts()

In [22]:
# Apply and expand "Dosage Form" data into two new columns
df["MediumVolume_Cleaned"] = df["Volume (mL)"].apply(extract_medium_volumes)
df = df.explode("MediumVolume_Cleaned", ignore_index=True)
# df["MediumVolume_Cleaned"].value_counts()

##### 3.6. Sampling times

Finally, the ………

In [23]:
# df[df.columns[6]].value_counts()
print(len(df))

2247


In [24]:
# Apply the function to your DataFrame
df["SamplingTimes_Cleaned"] = df["Recommended Sampling Times (minutes)"].apply(extract_sampling_times)

# Explode the list into separate rows
# df = df.explode("SamplingTimes_Cleaned", ignore_index=True)

# Now, you can use value_counts() to see the distribution
print(df["SamplingTimes_Cleaned"].value_counts())
print(len(df))

SamplingTimes_Cleaned
[nan]                                                                                                        767
[5.0, 10.0, 15.0, 20.0, 30.0]                                                                                138
[10.0, 20.0, 30.0, 45.0, 60.0]                                                                                87
[10.0, 15.0, 20.0, 30.0, 45.0]                                                                                80
[10.0, 20.0, 30.0, 45.0]                                                                                      76
[5.0, 10.0, 15.0, 20.0, 30.0, 45.0]                                                                           75
[120.0, 10.0, 20.0, 30.0, 45.0, 60.0]                                                                         45
[5.0, 10.0, 15.0, 30.0]                                                                                       26
[30.0, 60.0, 120.0, 15.0, 90.0, 180.0, 240.0]                             

In [25]:
df.head(15)

,Drug Name,Dosage Form,USP Apparatus,Speed (RPMs),Medium,Volume (mL),Recommended Sampling Times (minutes),Date Updated,DosageForm_Cleaned,ReleaseType_Cleaned,Apparatus_Cleaned,MediumType_Cleaned,MediumVolume_Cleaned,SamplingTimes_Cleaned
0,Abacavir Sulfate,Tablet,NaN,NaN,"Refer to FDA's Dissolution Guidance, 2018",NaN,NaN,07/02/2020,Tablet,None,NaN,NaN,NaN,[nan]
1,Abacavir Sulfate/Dolutegravir Sodium/Lamivudine,Tablet,II (Paddle),85,"0.01 M Phosphate Buffer with 0.5% sodium dodecyl sulfate (SDS), pH 6.8",900,"Abacavir and lamivudine: 10, 15, 20, 30 and 45; Dolutegravir: 5,15, 25, 35 and 45.",05/28/2015,Tablet,None,Paddle,phosphate_buffer,900,"[10.0, 15.0, 20.0, 30.0, 45.0, 5.0, 25.0, 35.0]"
2,Abacavir Sulfate/Dolutegravir Sodium/Lamivudine,Tablet,II (Paddle),85,"0.01 M Phosphate Buffer with 0.5% sodium dodecyl sulfate (SDS), pH 6.8",900,"Abacavir and lamivudine: 10, 15, 20, 30 and 45; Dolutegravir: 5,15, 25, 35 and 45.",05/28/2015,Tablet,None,Paddle,surfactant,900,"[10.0, 15.0, 20.0, 30.0, 45.0, 5.0, 25.0, 35.0]"
3,Abacavir Sulfate/Dolutegravir Sodium/Lamivudine,Tablet (For Suspension),II (Paddle),50,"0.01 M Phosphate Buffer with 0.5 mM EDTA, pH 6.8",500,"5, 10, 15, 30, 45 and 60",10/06/2023,Tablet,For Suspension,Paddle,phosphate_buffer,500,"[5.0, 10.0, 15.0, 30.0, 45.0, 60.0]"
4,Abacavir Sulfate/Lamivudine,Tablet,II (Paddle),75,0.1 N HCl,900,"10, 20, 30, and 45",01/03/2007,Tablet,None,Paddle,hcl,900,"[10.0, 20.0, 30.0, 45.0]"
5,Abacavir Sulfate/Lamivudine/Zidovudine,Tablet,II (Paddle),75,0.1 N HCl,Acid Stage: 900 mL; Buffer Stage: 1000 mL,"5, 10, 15, 30 and 45",01/03/2007,Tablet,None,Paddle,hcl,900,"[5.0, 10.0, 15.0, 30.0, 45.0]"
6,Abacavir Sulfate/Lamivudine/Zidovudine,Tablet,II (Paddle),75,0.1 N HCl,Acid Stage: 900 mL; Buffer Stage: 1000 mL,"5, 10, 15, 30 and 45",01/03/2007,Tablet,None,Paddle,hcl,1000,"[5.0, 10.0, 15.0, 30.0, 45.0]"
7,Abemaciclib,Tablet,II (Paddle),75,0.01 N HCl,900,"5, 10, 15, 20 and 30",11/16/2017,Tablet,None,Paddle,hcl,900,"[5.0, 10.0, 15.0, 20.0, 30.0]"
8,Abiraterone Acetate,Tablet,II (Paddle),50,"0.25% SLS in 56.5 mM phosphate buffer, pH 4.5",900,"10, 20, 30, 45 and 60",02/28/2013,Tablet,None,Paddle,phosphate_buffer,900,"[10.0, 20.0, 30.0, 45.0, 60.0]"
9,Abiraterone Acetate,Tablet,II (Paddle),50,"0.25% SLS in 56.5 mM phosphate buffer, pH 4.5",900,"10, 20, 30, 45 and 60",02/28/2013,Tablet,None,Paddle,surfactant,900,"[10.0, 20.0, 30.0, 45.0, 60.0]"


In [26]:
print(df.ReleaseType_Cleaned.value_counts())

ReleaseType_Cleaned
Extended Release                           377
Delayed Release                            241
Chewable                                    33
Orally Disintegrating                       27
Delayed Release, Orally Disintegrating      22
Extended Release, Orally Disintegrating     20
Copackage                                   12
Orally Disintegrating, Delayed Release      10
Sublingual                                  10
Buccal                                       9
Delayed Release Pellets                      6
Soft-Gelatin                                 6
Vaginal                                      5
Orally Disintegrating (ODT)                  4
For Suspension                               3
Soft-Gelatin/Liquid Fill                     2
Pediatric                                    2
For Oral Suspension                          2
Sprinkle                                     2
Effervescent                                 2
Liposomal                               

### 3'. Temporary Placeholder Heading

#### 3.1. Data Cleaning

Ensuring numerical features are clean and scaled.

Establishing known lists for reference in clean up.

In [27]:
# # Known "Dosage Form" release types to help detect them outside parentheses
# KNOWN_RELEASE_TYPES = [
#     "Extended Release", "Delayed Release", "Orally Disintegrating", 
#     "Immediate Release", "Controlled Release", "Sustained Release", 
#     "Copackage" # , "For Suspension"
# ]

# Known RPM units and possible variations
RPM_UNIT_MAP = {
    "RPM": ["rpm", "RPM", "revs/min", "revolutions per minute"],
    "DPM": ["dpm", "DPM", "dips/min", "dpm (dip rate per minute)"],
    "CPM": ["cpm", "CPM", "cycles per minute", "cycles/minute",
            "cycles/min", "cycles per min", "cycles (chews) per minute"],
}

# Flatten unit variants and build a regex alternation of *only* known units
UNIT_VARIANTS = sorted({v for vs in RPM_UNIT_MAP.values() for v in vs}, key=len, reverse=True)
UNIT_ALT = "|".join(re.escape(v) for v in UNIT_VARIANTS)  # longest-first avoids partial matches

# Creating tracking dictionary
rpm_case_tracking_dict = {
    "Case 1: empty entry": 0,
    "Case 2: pure number": 0,
    "Case 3: number + parenthetical note": 0,
    "Case 4: \"<Note>: <number> [unit?]\"": 0,
    "Case 5: \"X mg, Y mg and Z mg: N rpm\"": 0,
    "Case 6: number with unit embedded": 0,
    "Case 7: number + unit": 0,
    "Case 8: multiple numbers without unit": 0,
    "Fallback: keep as note": 0,
}

# Setting for which cases to print (for debugging)
rpm_case_printing_dict = {
    "Case 1: empty entry": False,
    "Case 2: pure number": False,
    "Case 3: number + parenthetical note": False,
    "Case 4: \"<Note>: <number> [unit?]\"": False,
    "Case 5: \"X mg, Y mg and Z mg: N rpm\"": False,
    "Case 6: number with unit embedded": False,
    "Case 7: number + unit": False,
    "Case 8: multiple numbers without unit": False,
    "Fallback: keep as note": False,
}

Defining cleanup functions.

In [28]:
# Defining function to cleanup "Dosage Form" data
def clean_dosage_form(value):
    """
    Separates raw "Dosage Form" data entries into "Dosage Form" and "Release Type".
    """
    if pd.isna(value):
        return ("Unknown", "Unknown")
    
    # Strip spaces
    val = str(value).strip()

    # # Look for release type in parentheses
    # m = re.match(r"^(.*?)\s*\((.*?)\)$", val)
    
    # Regex: capture base form + anything inside parentheses
    # e.g. "Tablet (Delayed Release, Orally Disintegrating)"
    m = re.match(r"^(.*?)\s*(?:\((.+)\))?$", val)
    # m = re.match(r"^([A-Za-z ]+?)(?:\s*\((.+)\))?$", val)
    if m:
        dosage_form = m.group(1).strip().title()  # normalize dosage form
        release_type = m.group(2).strip() if m.group(2) else None

        # Remove trailing commas from both
        dosage_form = dosage_form.rstrip(',').strip()
        dosage_form = re.sub(r"\s+", " ", dosage_form)
        if release_type:
            release_type = release_type.rstrip(',').strip() # Note: ODT refers to Orally Disintegrating Tablet
            release_type = re.sub(r"\s+", " ", release_type)
            if release_type and not release_type[0].isupper():
                release_type = release_type[0].upper() + release_type[1:]

        # If no parentheses captured and there’s a comma, check for trailing release type
        if not release_type and ',' in dosage_form:
            parts = [p.strip() for p in dosage_form.split(',')]
            # If last part looks like a known release type, separate it
            if parts[-1] in KNOWN_RELEASE_TYPES:
                release_type = parts[-1]
                dosage_form = ', '.join(parts[:-1]).rstrip(',').strip()
        
        return dosage_form, release_type
    else:
        # fallback: keep whole thing in base, no modifiers
        return val.rstrip(',').title(), None


# • Apparatus → categorical → one-hot encode
# • Medium type → categorical → one-hot encode
# • RPM → numerical
# • Medium volume → numerical
# • Sampling times → numerical summary features (e.g., # of samples, max time)

# Defining function to normalize "Speed (RPMs)" units
def normalize_unit(raw_unit: str) -> str:
    raw_unit = raw_unit.strip().lower()
    for canonical, variations in RPM_UNIT_MAP.items():
        for variation in variations:
            if raw_unit == variation.lower():
                return canonical
    return raw_unit  # fallback if no match

# Defining function to cleanup "Speed (RPMs)" data
def clean_speed_rpms(entry):
    """
    Returns a list of tuples: (Speed, Unit, Notes)
    Handles multiple numbers per entry and colon/semicolon-separated cases.
    """
    # Case 1: empty entry
    if pd.isna(entry):
        rpm_case_tracking_dict["Case 1: empty entry"] += 1
        if rpm_case_printing_dict["Case 1: empty entry"] == True:
            print("Case 1: empty entry                  ", (None, None, None))
        return [(None, None, None)]

    # Replacing any Unicode non-breaking space characters ("\xa0") with spaces
    # val = str(entry).replace("\xa0", " ").strip()
    
    # Replace non-breaking spaces and collapse all whitespace in one go
    val = re.sub(r'\s+', ' ', str(entry).replace("\xa0", " ")).strip()

    results = []

    # Split by semicolons for multiple measurements
    parts = [p.strip() for p in val.split(";")]

    for part in parts:
        # Case 2: pure number
        if part.isdigit():
            results.append((int(part), "RPM", None))
            rpm_case_tracking_dict["Case 2: pure number"] += 1
            if rpm_case_printing_dict["Case 2: pure number"] == True:
                print("Case 2: pure number                  ", (int(part), "RPM", None))
            continue

        # Case 3: number + parenthetical note
        if re.search(r"\d+\s*\([^)]*\)", part):
            # Split on "&" inside this case, because it can contain multiple entries
            subparts = [subpart.strip() for subpart in re.split(r"&", part)]
            for subpart in subparts:
                match = re.match(r"(\d+)\s*\(([^)]*)\)", subpart)
                if match:
                    speed = int(match.group(1))
                    note = match.group(2).strip()
                    results.append((speed, "RPM", note))
                    rpm_case_tracking_dict["Case 3: number + parenthetical note"] += 1
                    if rpm_case_printing_dict.get("Case 3: number + parenthetical note", True):
                        print("Case 3: number + parenthetical note  ", (speed, "RPM", note))
            continue

        # Case 4: "<Note>: <number> [unit?]"
        colon_match = re.match(r"(.+?):\s*(\d+)\s*([A-Za-z/]+)?", part)
        if colon_match:
            note = colon_match.group(1).strip()
            speed = int(colon_match.group(2))
            unit = colon_match.group(3).lower() if colon_match.group(3) else None
            raw_unit = colon_match.group(3)
            
            if raw_unit:
                mapped_unit = RPM_UNIT_MAP.get(raw_unit.lower())
                if mapped_unit:  
                    # Normal case: recognized unit
                    unit = mapped_unit
                else:
                    # Fallback: treat as note addition
                    unit = "RPM"
                    extra_note = raw_unit.capitalize()
                    note = f"{note} ({extra_note})"
            else:
                unit = "RPM"

            # Remove any additional colons from the note
            note = note.replace(":", "").strip()
            
            results.append((speed, unit, note))
            rpm_case_tracking_dict["Case 4: \"<Note>: <number> [unit?]\""] += 1
            if rpm_case_printing_dict["Case 4: \"<Note>: <number> [unit?]\""]:
                print("Case 4: \"<Note>: <number> [unit?]\"   ", (speed, unit, note))
            continue

        # Case 5: "X mg, Y mg and Z mg: N rpm"
        multi_match = re.match(r"^\s*([^\:]+):\s*(\d+)\s*([A-Za-z/]+)?", part)
        if multi_match:
            note = multi_match.group(1).strip()  # everything before the ":"
            note = re.sub(r':$', '', note).strip()  # drop trailing colon if present
            
            speed = int(multi_match.group(2))
            unit = multi_match.group(3).lower() if multi_match.group(3) else "rpm"
            unit = RPM_UNIT_MAP.get(unit.lower(), unit.upper())
        
            results.append((speed, unit, note))
        
            rpm_case_tracking_dict["Case 5: \"X mg, Y mg and Z mg: N rpm\""] += 1
            if rpm_case_printing_dict["Case 5: \"X mg, Y mg and Z mg: N rpm\""]:
                print(part)
                print("Case 5: \"X mg, Y mg and Z mg: N rpm\" ", (speed, unit, note))
            continue

        # Case 6: number with recognized unit anywhere in the string
        # 6a) Hyphen/and/to ranges with a recognized unit → average
        m_range = re.search(rf'\b(\d+)\s*(?:-|and|to)\s*(\d+)\s*({UNIT_ALT})\b', part, flags=re.I)
        if m_range:
            a, b = int(m_range.group(1)), int(m_range.group(2))
            unit_text = m_range.group(3)
            unit = normalize_unit(unit_text)  # canonical like "RPM"/"DPM"/"CPM"
            avg = (a + b) // 2 if (a + b) % 2 == 0 else (a + b) / 2
        
            # remove the matched segment (optionally preceded by "at")
            note = re.sub(
                rf'(\bat\s+)?\b{a}\s*(?:-|and|to)\s*{b}\s*{re.escape(unit_text)}\b',
                '',
                part,
                flags=re.I,
                count=1
            ).strip(",;: ")
        
            # tidy note: collapse "at for" → "for" and extra spaces
            note = re.sub(r'\bat\s+for\b', 'for', note, flags=re.I)
            note = re.sub(r'\s{2,}', ' ', note).strip() or None
        
            results.append((int(avg), unit, note))
            rpm_case_tracking_dict["Case 6: number with unit embedded"] += 1
            if rpm_case_printing_dict["Case 6: number with unit embedded"]:
                print("Case 6: number with unit embedded  * ", results[-1])
            continue
        
        # 6b) Single number + recognized unit
        m_single = re.search(rf'\b(\d+)\s*({UNIT_ALT})\b', part, flags=re.I)
        if m_single:
            speed = int(m_single.group(1))
            unit_text = m_single.group(2)
            unit = normalize_unit(unit_text)
        
            note = re.sub(
                rf'(\bat\s+)?\b{speed}\s*{re.escape(unit_text)}\b',
                '',
                part,
                flags=re.I,
                count=1
            ).strip(",;: ")
        
            note = re.sub(r'\bat\s+for\b', 'for', note, flags=re.I)
            note = re.sub(r'\s{2,}', ' ', note).strip() or None
        
            results.append((speed, unit, note))
            rpm_case_tracking_dict["Case 6: number with unit embedded"] += 1
            if rpm_case_printing_dict["Case 6: number with unit embedded"]:
                print("Case 6: number with unit embedded  * ", results[-1])
            continue

        # Case 7: number + unit
        match = re.match(r"^(\d+)\s*([A-Za-z/ ]+)$", part)
        if match:
            speed = int(match.group(1))
            unit_raw = match.group(2).strip().lower()
        
            # Only accept units we know
            if unit_raw in RPM_UNIT_MAP:
                unit_norm = normalize_unit(unit_raw)
                # unit = RPM_UNIT_MAP[unit_raw]
                results.append((speed, unit_norm, None))
                rpm_case_tracking_dict["Case 7: number + unit"] += 1
                if rpm_case_printing_dict["Case 7: number + unit"]:
                    print("Case 7: number + unit     * * *      ", (speed, unit_norm, None))
            else:
                # Otherwise treat the whole string as a note
                results.append((None, None, part))
                rpm_case_tracking_dict["Fallback: keep as note"] += 1
                if rpm_case_printing_dict["Fallback: keep as note"]:
                    print("Fallback: keep as note               ", (None, None, part))
            continue

        # Case 8: multiple numbers with "and" or "-" but no unit
        m_multi = re.match(r'^\s*(\d+)\s*(?:and|-|to)\s*(\d+)\s*$', part, flags=re.I)
        if m_multi:
            a, b = int(m_multi.group(1)), int(m_multi.group(2))
            # emit two rows, one for each number
            results.append((a, "RPM", None))
            results.append((b, "RPM", None))
            rpm_case_tracking_dict["Case 8: multiple numbers without unit"] += 1
            if rpm_case_printing_dict["Case 8: multiple numbers without unit"]:
                print("Case 8: multiple numbers without unit", (a, "RPM", None))
                print("Case 8: multiple numbers without unit", (b, "RPM", None))
            continue

        # Fallback: keep as note
        results.append((None, None, part))
        rpm_case_tracking_dict["Fallback: keep as note"] += 1
        if rpm_case_printing_dict["Fallback: keep as note"] == True:
            print("Fallback: keep as note               ", (None, None, part))
            
    return results



# Defining function to cleanup "Medium" data
def clean_medium(entry):
    pass

# Defining function to cleanup "Volume (mL)" data

# Defining function to cleanup "Recommended Sampling Times (minutes)" data

In [29]:
# # Replacing any Unicode non-breaking space characters ("\xa0") with spaces
# val = str(df["Speed (RPMs) (Raw)"]).replace("\xa0", " ").strip()
# results = []

# # Split by semicolons for multiple measurements
# parts = [p.strip() for p in val.split(";")]

# # Define regex patterns to explore
# rpm_regex_patterns = {
#     "digits_only": r'^\d+$',               # pure digits only
#     "rpm_values": r'^\d+\s?RPM$',          # e.g., '50 RPM'
#     "dpm_values": r'^\d+\s?DPM$',          # e.g., '30 DPM'
#     "unclear_values": r'(\d+)\s*([A-Za-z/ ]+)',
#     "embedded_cycles": r'(\d+)\s*(rpm|dpm|cycles per min|cycles/min)',
#     "drug_based": r'([\d, andmgMG ]+):\s*(\d+)\s*([A-Za-z/]+)?',
#     "other": r'(.+?):\s*(\d+)\s*([A-Za-z/]+)?',
# }

# # Function to match each semicolon-separated part
# def match_parts(entry, pattern):
#     entry = str(entry).replace("\xa0", " ").strip()
#     parts = [p.strip() for p in entry.split(";")]
#     matched_parts = [p for p in parts if pattern.match(p)]
#     return matched_parts if matched_parts else [None]

# # Choose a pattern to display (change this key as needed)
# selected_rpm_pattern_key = "embedded_cycles"
# selected_rpm_pattern = rpm_regex_patterns[selected_rpm_pattern_key]

# # Compile regex
# pattern = re.compile(selected_rpm_pattern, flags=re.IGNORECASE)

# # Filter rows that MATCH this pattern
# mask = df["Speed (RPMs) (Raw)"].astype(str).str.match(pattern)

# # Compute value counts DataFrame
# vc = df.loc[mask, "Speed (RPMs) (Raw)"].value_counts().to_frame(name="count")

# # Count how many rows will be displayed
# row_count = len(vc)
# print(f"Number of rows displayed for pattern '{selected_rpm_pattern_key}': {row_count}")

# # Apply styling for display in Jupyter
# filtered = (
#     vc.style
#     .set_table_attributes('style="display:inline"')
#     .set_table_styles(
#         [{'selector': 'table',
#           'props': [('max-height', '400px'),
#                     ('overflow-y', 'scroll'),
#                     ('display', 'block')]}]
#     )
# )

# filtered

In [30]:
# unique_values = pd.DataFrame(df["Speed (RPMs) (Raw)"].unique(), columns=["Unique RPMs"])

# # Force display of all rows in this DataFrame
# with pd.option_context("display.max_rows", None):
#     display(unique_values)

Running functions to clean up dataset.

In [31]:
# # # Apply and expand "Dosage Form" data into two new columns
# # df[["Dosage Form (Clean)", "Release Type"]] = df["Dosage Form"].apply(lambda x: pd.Series(clean_dosage_form(x)))
# # expanded_test
# # # # Check results
# # # print(df['DosageForm_clean'].value_counts())
# # # print(df['ReleaseType'].value_counts())



# # Preparing for cleanup of "Speed (RPMs)" data
# df.rename(columns={"Speed (RPMs)": "Speed (RPMs) (Raw)"}, inplace=True)

# # df["Speed (RPMs) (Raw)"].value_counts().reset_index(name="count").rename(columns={"index": "Speed (RPMs) (Raw)"})
# num_unique_rpms = len(df["Speed (RPMs) (Raw)"].value_counts())
# print("Number of rows displayed:", num_unique_rpms)

# # df["Speed (RPMs) (Raw)"].value_counts().to_frame().style.set_table_attributes('style="display:inline"').set_table_styles(
# #     [{'selector': 'table', 'props': [('max-height', '400px'), ('overflow-y', 'scroll'), ('display', 'block')]}]
# # )



# # Apply and expand, safely overwriting existing columns
# expanded_rows = df["Speed (RPMs) (Raw)"].apply(clean_speed_rpms)
# df_cleaned = pd.DataFrame(
#     [t for sublist in expanded_rows for t in sublist],
#     columns=["Speed (RPMs) (Clean)", "Unit", "Notes"]
# )

# # Overwrite the old columns safely
# df[["Speed (RPMs) (Clean)", "Unit", "Notes"]] = df_cleaned

# pd.DataFrame(list(rpm_case_tracking_dict.items()), columns=["Case", "Count"])

# # df[df["Speed (RPMs) (Raw)"] == "50 RPM (12.5 mg, 25 mg and 100 mg); 75 RPM (150 mg and 200 mg)"]
# # df[-200:]








# # # Example transformations
# # # Performing dataset cleanup as needed before one-hot encoding
# # df['num_sampling_times'] = df['SamplingTimes'].apply(lambda x: len(str(x).split(',')))
# # df['max_sampling_time'] = df['SamplingTimes'].apply(lambda x: max([int(t) for t in str(x).split(',')]))

# # # One-hot encode categorical variables
# # df_encoded = pd.get_dummies(df[['Apparatus', 'MediumType', 'DosageForm']])

# # # Combine numeric + encoded
# # features = pd.concat([df[['RPM', 'MediumVolume', 'num_sampling_times', 'max_sampling_time']], df_encoded], axis=1)


# # # Drop non-numeric columns if needed after feature engineering is complete (sanity check)
# # df = df.select_dtypes(include=[np.number]).dropna()

# # features.head()

#### 3.2. Preprocessing

* Apparatus → categorical → one-hot encode
* Medium type → categorical → one-hot encode
* RPM → numerical
* Medium volume → numerical
* Sampling times → numerical summary features (e.g., # of samples, max time)

In this case, all of the columns except the target column are going to be used to predict the target column.

In other words, using a patient's medical and demographic data to predict whether or not they have heart disease.

#### 3.3. Feature Engineering

In this case, a selection of synthetic dissolution profiles will be simulated from the FDA metadata (dosage form, apparatus type, agitation speed (rpm), medium composition, medium volume, and sampling times). For instance, a first-order or Weibull kinetic model may be used to generate curves plotting the percentage of the drug dissolved over time. Parameters can parhaps be linked to method settings (e.g., higher RPM values could lead to faster rate constants).

After this, it will be possible to use GMMs within a Machine Learning workflow and a patient's medical and demographic data to predict whether or not they have heart disease. !!!!!!!!!!!!!!!!!!!!!!

Convert the FDA method metadata into ML-ready features:
- Apparatus → categorical → one-hot encode  
- Medium type → categorical → one-hot encode  
- RPM → numerical  
- Medium volume → numerical  
- Sampling times → numerical summary features (e.g., # of samples, max time) 

In [32]:
# # Example transformations
# df['num_sampling_times'] = df['SamplingTimes'].apply(lambda x: len(str(x).split(',')))
# df['max_sampling_time'] = df['SamplingTimes'].apply(lambda x: max([int(t) for t in str(x).split(',')]))

# # One-hot encode categorical variables
# df_encoded = pd.get_dummies(df[['Apparatus','MediumType','DosageForm']])

# # Combine numeric + encoded
# features = pd.concat([df[['RPM','MediumVolume','num_sampling_times','max_sampling_time']], df_encoded], axis=1)

# features.head()

1. Problem defintion
2. Data
3. Evaluation
4. Features
5. Modelling
6. Experiments

#### 3.9 ???? Creating Cleaned Dataset

In [33]:
dissolution_data_cleaned_df = df.drop([], axis=1)

### 4. Calculating Missing Data (Synthetic Profile Generation)

Simulating % dissolved vs. time data using kinetic models:

- **First-order model:**  
  C(t) = 100 * (1 - exp(-k * t))  
- **Weibull model:**  
  C(t) = 100 * (1 - exp(-(t/τ)^β))  

Linking `k` or `(τ, β)` parameters loosely to method features (e.g., higher RPM → faster dissolution).

In [34]:
# def simulate_first_order(t_points, k):
#     return 100 * (1 - np.exp(-k * t_points))

# # Example: generate synthetic curves for each FDA entry
# t_points = np.linspace(0, 60, 20)  # 0–60 min, 20 points
# synthetic_profiles = []

# for i, row in df.iterrows():
#     rpm = row['RPM']
#     # crude mapping: higher RPM = faster rate
#     k = 0.01 + (rpm / 500) * 0.1
#     profile = simulate_first_order(t_points, k)
#     synthetic_profiles.append(profile)

# synthetic_profiles = np.array(synthetic_profiles)

#### 4.1. Condensing Full DataFrame into a Simpler and Cleaned DataFrame

In [35]:
# Renaming 'Drug Name' column header foer consistency
df = df.rename(columns={"Drug Name": "DrugName"})

# Define the columns you want to keep
columns_to_keep = [
    "DrugName",
    "DosageForm_Cleaned",
    "ReleaseType_Cleaned",
    "Apparatus_Cleaned",
    "SpeedRPM_Cleaned",
    "MediumType_Cleaned",
    "MediumVolume_Cleaned",
    "SamplingTimes_Cleaned"
]

# Create a cleaned copy
# df_cleaned = df[columns_to_keep].copy()
df_cleaned = df.loc[:, df.columns.intersection(columns_to_keep)].copy()

# Check the result
df_cleaned.head()

,DrugName,DosageForm_Cleaned,ReleaseType_Cleaned,Apparatus_Cleaned,MediumType_Cleaned,MediumVolume_Cleaned,SamplingTimes_Cleaned
0,Abacavir Sulfate,Tablet,None,NaN,NaN,NaN,[nan]
1,Abacavir Sulfate/Dolutegravir Sodium/Lamivudine,Tablet,None,Paddle,phosphate_buffer,900,"[10.0, 15.0, 20.0, 30.0, 45.0, 5.0, 25.0, 35.0]"
2,Abacavir Sulfate/Dolutegravir Sodium/Lamivudine,Tablet,None,Paddle,surfactant,900,"[10.0, 15.0, 20.0, 30.0, 45.0, 5.0, 25.0, 35.0]"
3,Abacavir Sulfate/Dolutegravir Sodium/Lamivudine,Tablet,For Suspension,Paddle,phosphate_buffer,500,"[5.0, 10.0, 15.0, 30.0, 45.0, 60.0]"
4,Abacavir Sulfate/Lamivudine,Tablet,None,Paddle,hcl,900,"[10.0, 20.0, 30.0, 45.0]"


#### 4.2. Simulating Dissolution Curves Using Cleaned DataFrame

In [36]:
# Testing: Simulating Dissolution Curve for first row of dataset
row = df_cleaned.iloc[0]
df_curve = simulate_dissolution_curve(row)
print(df_curve.head(15))

    Time_min  Dissolved_pct
0          0            NaN
1          5            NaN
2         10            NaN
3         15            NaN
4         20            NaN
5         25            NaN
6         30            NaN
7         35            NaN
8         40            NaN
9         45            NaN
10        50            NaN
11        55            NaN
12        60            NaN
13        65            NaN
14        70            NaN


In [37]:
# --- 1. Simulate Dissolution Curves for All Rows ---

# Duplicate cleaned DataFrame for manipulation
df_cleaned_curves = df_cleaned.copy()

# # Generate and store curves
# df_cleaned_curves["DissolutionProfile"] = df_cleaned_curves.apply(
#     lambda row: simulate_dissolution_curve(row)["Dissolved_pct"].values, axis=1
# )

# # Store time points (same for all)
# time_points = simulate_dissolution_curve(df_cleaned_curves.iloc[0])["Time_min"].values
# df_cleaned_curves["SamplingTimes_Cleaned"] = [time_points] * len(df_cleaned_curves)

# # Preview the result
# print(df_cleaned_curves[["DrugName", "ReleaseType_Cleaned", "DissolutionProfile"]].head())

# # # Simulate dissolution curves for all rows and store in a new column
# # df_cleaned["DissolutionCurve"] = df_cleaned.apply(
# #     simulate_dissolution_curve,
# #     axis=1
# # )

# Example: Apply to all rows, using SamplingTimes_Cleaned if available
df_cleaned_curves["DissolutionProfile"] = df_cleaned_curves.apply(
    lambda row: simulate_dissolution_curve(
        row,
        time_points=row.get("SamplingTimes_Cleaned")
    )["Dissolved_pct"].values,
    axis=1
)

# Store the time points (which may vary per row)
df_cleaned_curves["SamplingTimes_Used"] = df_cleaned_curves.apply(
    lambda row: row.get("SamplingTimes_Cleaned", np.arange(0, 125, 5)),
    axis=1
)

                                          DrugName ReleaseType_Cleaned  \
0                                 Abacavir Sulfate                None   
1  Abacavir Sulfate/Dolutegravir Sodium/Lamivudine                None   
2  Abacavir Sulfate/Dolutegravir Sodium/Lamivudine                None   
3  Abacavir Sulfate/Dolutegravir Sodium/Lamivudine      For Suspension   
4                      Abacavir Sulfate/Lamivudine                None   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  DissolutionProfile  
0                                                                   

In [38]:
# --- 2. Plot Dissolution Curves ---

# Defining function for plotting dissolution curves
def plot_dissolution_curves(df, group_col="DrugName"):
    """Plot all simulated dissolution curves, grouped and colored by `group_col`."""
    plt.figure(figsize=(10, 6))
    
    # Assign unique colors per group
    groups = df[group_col].unique()
    cmap = plt.cm.get_cmap("tab10", len(groups))
    colors = dict(zip(groups, cmap.colors))

    for _, row in df.iterrows():
        plt.plot(
            row["SamplingTimes_Cleaned"],
            row["DissolutionProfile"],
            color=colors.get(row[group_col], "gray"),
            label=row[group_col],
            alpha=0.6
        )

    plt.xlabel("Time (min)")
    plt.ylabel("Percent Dissolved (%)")
    plt.title(f"Dissolution Curves by {group_col}")
    
    # Avoid duplicate labels
    handles, labels = plt.gca().get_legend_handles_labels()
    unique = dict(zip(labels, handles))
    plt.legend(unique.values(), unique.keys(), bbox_to_anchor=(1.05, 1), loc='upper left', title=group_col)
    
    plt.tight_layout()
    plt.show()

# # Concatenate all curves into a single DataFrame for analysis/plotting
# all_curves = []
# for i, curve in enumerate(df_cleaned["DissolutionCurve"]):
#     drug_name = df_cleaned.loc[i, "DrugName"]
#     release_type = df_cleaned.loc[i, "ReleaseType_Cleaned"]
#     temp_curve = curve.copy()
#     temp_curve["DrugName"] = drug_name
#     temp_curve["ReleaseType"] = release_type
#     all_curves.append(temp_curve)

# df_all_curves = pd.concat(all_curves, ignore_index=True)

# # Concatenate all curves into a single DataFrame for analysis/plotting
# all_curves = []
# for i, curve in enumerate(df_cleaned["DissolutionCurve"]):
#     curve["DrugName"] = df_cleaned.loc[i, "DrugName"]
#     curve["ReleaseType"] = df_cleaned.loc[i, "ReleaseType_Cleaned"]
#     all_curves.append(curve)

# df_all_curves = pd.concat(all_curves, ignore_index=True)


# # # Concatenating all curves
# # df_all_curves = pd.concat(
# #     [
# #         pd.concat([
# #             pd.DataFrame({"Index": [i] * len(curve),
# #                           "DrugName": [df_cleaned_curves.loc[i, "DrugName"]] * len(curve)}),
# #             curve.reset_index(drop=True)
# #         ], axis=1)
# #         for i, curve in enumerate(df_cleaned_curves["DissolutionCurve"])
# #     ],
# #     ignore_index=True
# # )

In [39]:
# # --- 3. Example Usage ---

# # Plot by drug
# plot_dissolution_curves(df_cleaned_curves, group_col="DrugName")

# # Plot by release type
# plot_dissolution_curves(df_cleaned_curves, group_col="ReleaseType_Cleaned")

In [40]:
# def plot_dissolution_curves(df, group_col="DrugName"):
#     """Plot all dissolution curves in df grouped by a column (e.g., DrugName or ReleaseType_Cleaned)."""
#     plt.figure(figsize=(10, 6))
    
#     for _, row in df.iterrows():
#         plt.plot(row["SamplingTimes_Cleaned"], row["DissolutionProfile"], 
#                  label=row[group_col], alpha=0.6)
    
#     plt.xlabel("Time (min)")
#     plt.ylabel("Percent Dissolved (%)")
#     plt.title(f"Dissolution Curves by {group_col}")
#     plt.legend(title=group_col, bbox_to_anchor=(1.05, 1), loc='upper left')
#     plt.tight_layout()
#     plt.show()

In [41]:
print(df_cleaned.columns)

Index(['DrugName', 'DosageForm_Cleaned', 'ReleaseType_Cleaned',
       'Apparatus_Cleaned', 'MediumType_Cleaned', 'MediumVolume_Cleaned',
       'SamplingTimes_Cleaned'],
      dtype='object')


In [42]:
df_cleaned_curves = df_cleaned.copy()

# Generate simulated dissolution curves for each row
dissolution_profiles = []
for _, row in df_cleaned_curves.iterrows():
    curve_df = simulate_dissolution_curve(row)
    dissolution_profiles.append(curve_df["Dissolved_pct"].values)  # store just % dissolved

# Add the simulated profiles back into the DataFrame
df_cleaned_curves["DissolutionProfile"] = dissolution_profiles

# Optionally, store time points too (they're all the same anyway)
df_cleaned_curves["SamplingTimes_Cleaned"] = [simulate_dissolution_curve(df_cleaned_curves.iloc[0])["Time_min"].values] * len(df_cleaned_curves)



# Plot by drug
plot_dissolution_curves(df_cleaned, group_col="DrugName")

# Plot by release type
plot_dissolution_curves(df_cleaned, group_col="ReleaseType_Cleaned")

/var/folders/95/8jvmv01x473b_h3h3fd9ml580000gn/T/ipykernel_10151/694687538.py:10: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = plt.cm.get_cmap("tab10", len(groups))


KeyError: 'DissolutionProfile'

<Figure size 1000x600 with 0 Axes>

### 5. Training the Data Using Gaussian Mixture Model Clustering

Clustering the simulated profiles to identify subgroups:
- Input: synthetic profile matrix
- Scaling applied
- Model selection with AIC/BIC

In [ ]:
# Standardize data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(synthetic_profiles)

# Try different numbers of clusters
bic_scores = []
aic_scores = []
n_components = range(1, 6)

for n in n_components:
    gmm = GaussianMixture(n_components=n, random_state=42)
    gmm.fit(X_scaled)
    bic_scores.append(gmm.bic(X_scaled))
    aic_scores.append(gmm.aic(X_scaled))

plt.plot(n_components, bic_scores, label="BIC")
plt.plot(n_components, aic_scores, label="AIC")
plt.xlabel("Number of clusters")
plt.ylabel("Score")
plt.legend()
plt.show()

In [ ]:
# Fit best GMM (say 3 clusters)
best_n = 3
gmm = GaussianMixture(n_components=best_n, random_state=42)
labels = gmm.fit_predict(X_scaled)

# Attach clusters
df['Cluster'] = labels

In [ ]:
# Create X (all the relevant feature columns)
X = dissolution_data_cleaned_df.drop("target", axis=1)

# Create y (the target column)
y = dissolution_data_cleaned_df["target"]

In [ ]:
# Split the data into training and test sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

# View the data shapes
X_train.shape, X_test.shape, y_train.shape, y_test.shape

### 6. Choosing the model/estimator

Scikit-Learn's machine learning map can usually be employed at this stage ([Scikit-Learn machine learning map](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html)).

<img src="../images/sklearn-ml-map.png" width=500/>

In Scikit-Learn, machine learning models are referred to as estimators.

In this case, since we're working on a ??? classification problem, we've chosen the RandomForestClassifier estimator which is part of the ensembles module.

Gaussian Mixture Model (GMM) !!!!!

In [ ]:
# Importing GMM model
from sklearn.ensemble import RandomForestClassifier # !!!
model = RandomForestClassifier()

### 7. Fit the model to the data and use it to make a prediction

The Gaussian Mixture Model will attempt to learn the patterns in the dataset by calling the `fit()` function on it and passing it the data.

In [ ]:
model.fit(X_train, y_train)

Once a model has learned patterns in data, you can use them to make a prediction with the `predict()` function.

In [ ]:
# Make predictions
y_preds = model.predict(X_test)

In [ ]:
# This will be in the same format as y_test
y_preds

In [ ]:
X_test.loc[206]

In [ ]:
heart_disease.loc[206]

In [ ]:
# Make a prediction on a single sample (has to be array)
model.predict(np.array(X_test.loc[206]).reshape(1, -1))

### 8. Evaluate the Model

A trained model/estimator can be evaluated by calling the `score()` function and passing it a collection of data.

In [ ]:
# On the training set
model.score(X_train, y_train)

In [ ]:
# On the test set (unseen)
model.score(X_test, y_test)

### 9. Experiment to Improve (Hyperparameter Tuning)

Since a model's first evaluation metrics are usually not its last, one method to improve the model's predictions can be with hyperparameter tuning.

In [ ]:
# Try different numbers of estimators (n_estimators is a hyperparameter you can change)
np.random.seed(42)
for i in range(10, 100, 10):
    print(f"Trying model with {i} estimators...")
    model = RandomForestClassifier(n_estimators=i).fit(X_train, y_train)
    print(f"Model accruacy on test set: {model.score(X_test, y_test)}")
    print("")

**Note:** It's best practice to test different hyperparameters with a validation set or cross-validation.

In [ ]:
from sklearn.model_selection import cross_val_score

# Try different numbers of estimators with cross-validation and no cross-validation
np.random.seed(42)
for i in range(10, 100, 10):
    print(f"Trying model with {i} estimators...")
    model = RandomForestClassifier(n_estimators=i).fit(X_train, y_train)
    print(f"Model accruacy on test set: {model.score(X_test, y_test)}")
    print(f"Cross-validation score: {np.mean(cross_val_score(model, X, y, cv=5)) * 100}%")
    print("")

### 10. Save the Model for Later Use

A trained model can be exported and saved so it can be imported and used later. One way to save a model is using Python's `pickle` module.

In [ ]:
import pickle

# Save trained model to file
pickle.dump(model, open("random_forest_model_1.pkl", "wb"))

In [ ]:
# Load a saved model and make a prediction on a single example
loaded_model = pickle.load(open("random_forest_model_1.pkl", "rb"))
loaded_model.predict(np.array(X_test.loc[206]).reshape(1, -1))

# A Simple Scikit-Learn Classification Workflow

This notebook shows a breif workflow you might use with `scikit-learn` to build a machine learning model to classify whether or not a patient has heart disease.

It follows the diagram below:

<img src="../images/sklearn-workflow.png"/>

**Note:** This workflow assumes your data is ready to be used with machine learning models (is numerical, has no missing values).

In [ ]:
# Standard imports
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

## 1. Get the data ready

In [ ]:
# Import dataset
heart_disease = pd.read_csv("../data/heart-disease.csv")

# View the data
heart_disease.head()

With this example, we're going to use all of the columns except the target column to predict the targert column.

In other words, using a patient's medical and demographic data to predict whether or not they have heart disease.

In [ ]:
# Create X (all the feature columns)
X = heart_disease.drop("target", axis=1)

# Create y (the target column)
y = heart_disease["target"]

In [ ]:
# Split the data into training and test sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

# View the data shapes
X_train.shape, X_test.shape, y_train.shape, y_test.shape

## 2. Choose the model/estimator

You can do this using the [Scikit-Learn machine learning map](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html).

<img src="../images/sklearn-ml-map.png" width=500/>

In Scikit-Learn, machine learning models are referred to as estimators.

In this case, since we're working on a classification problem, we've chosen the RandomForestClassifier estimator which is part of the ensembles module.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()

## 3. Fit the model to the data and use it to make a prediction

A model will (attempt to) learn the patterns in a dataset by calling the `fit()` function on it and passing it the data.

In [ ]:
model.fit(X_train, y_train)

Once a model has learned patterns in data, you can use them to make a prediction with the `predict()` function.

In [ ]:
# Make predictions
y_preds = model.predict(X_test)

In [ ]:
# This will be in the same format as y_test
y_preds

In [ ]:
X_test.loc[206]

In [ ]:
heart_disease.loc[206]

In [ ]:
# Make a prediction on a single sample (has to be array)
model.predict(np.array(X_test.loc[206]).reshape(1, -1))

## 4. Evaluate the model
A trained model/estimator can be evaluated by calling the `score()` function and passing it a collection of data.

In [ ]:
# On the training set
model.score(X_train, y_train)

In [ ]:
# On the test set (unseen)
model.score(X_test, y_test)

## 5. Experiment to improve (hyperparameter tuning)

A model's first evaluation metrics aren't always its last. One way to improve a models predictions is with hyperparameter tuning.

In [ ]:
# Try different numbers of estimators (n_estimators is a hyperparameter you can change)
np.random.seed(42)
for i in range(10, 100, 10):
    print(f"Trying model with {i} estimators...")
    model = RandomForestClassifier(n_estimators=i).fit(X_train, y_train)
    print(f"Model accruacy on test set: {model.score(X_test, y_test)}")
    print("")

**Note:** It's best practice to test different hyperparameters with a validation set or cross-validation.

In [ ]:
from sklearn.model_selection import cross_val_score

# Try different numbers of estimators with cross-validation and no cross-validation
np.random.seed(42)
for i in range(10, 100, 10):
    print(f"Trying model with {i} estimators...")
    model = RandomForestClassifier(n_estimators=i).fit(X_train, y_train)
    print(f"Model accruacy on test set: {model.score(X_test, y_test)}")
    print(f"Cross-validation score: {np.mean(cross_val_score(model, X, y, cv=5)) * 100}%")
    print("")

## 6. Save a model for later use

A trained model can be exported and saved so it can be imported and used later. One way to save a model is using Python's `pickle` module.

In [ ]:
import pickle

# Save trained model to file
pickle.dump(model, open("random_forest_model_1.pkl", "wb"))

In [ ]:
# Load a saved model and make a prediction on a single example
loaded_model = pickle.load(open("random_forest_model_1.pkl", "rb"))
loaded_model.predict(np.array(X_test.loc[206]).reshape(1, -1))